In [205]:
import syft as sy
from syft.spdz import spdz
from syft.mpc.securenn import decompose, private_compare, msb, relu_deriv
from syft.core.frameworks.torch.tensor import _GeneralizedPointerTensor, _SPDZTensor, _SNNTensor
from syft.core.frameworks.torch import utils as torch_utils

import unittest
import numpy as np
import torch
import importlib

hook = sy.TorchHook(verbose=True)

me = hook.local_worker
me.is_client_worker = False

bob = sy.VirtualWorker(id="bob", hook=hook, is_client_worker=False)
alice = sy.VirtualWorker(id="alice", hook=hook, is_client_worker=False)

me.add_workers([bob, alice])
bob.add_workers([me, alice])
alice.add_workers([me, bob])

In [277]:
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

def modinv(a, m):
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        return x % m

In [278]:
spdz.field

2147483648

In [281]:
modinv(5, spdz.field)

1288490189

In [283]:
((10) * 1288490189) % spdz.field

2

In [269]:
x = torch.FloatTensor([10]).fix_precision()

In [270]:
(x.child.child * modinv(101, spdz.field)) % spdz.field


 2.0837e+09
[syft.core.frameworks.torch.tensor.LongTensor of size 1]

In [223]:
x = torch.FloatTensor([-1,-2,3]).fix_precision()
BASE = 10
PRECISION = x.child.precision_fractional
PRECISION_INTEGRAL = 3

a = x.share(alice,bob)
a = ((a.child.child * a.child.child))
b = a + (BASE**(2*PRECISION+1))

In [222]:



# TODO: generate this mask on P0
mask = (torch.rand(list(a.get_shape())) * spdz.field).long()
mask_low = mask % (BASE**PRECISION)
b_masked = (b + mask.share(alice,bob)).get()
b_masked_low = b_masked % (BASE**PRECISION)
b_low = b_masked_low.share(bob, alice) - mask_low.share(alice, bob)
c = a - b_low

In [218]:
(c.child.child / 1000).sum_get()


 2.1485e+06
 2.1515e+06
 2.1565e+06
[syft.core.frameworks.torch.tensor.LongTensor of size 3]

In [216]:
c.get()


 1.0000e+06
 3.9996e+06
 8.9996e+06
[syft.core.frameworks.torch.tensor.LongTensor of size 3]

In [210]:
x

[Fixed precision]

-1
-2
 3
[syft.core.frameworks.torch.tensor.FloatTensor of size 3]

In [2]:
x = (torch.LongTensor([[-1,3,-5,7],[-1,0,1,2]])).share(alice,bob)
y = (torch.LongTensor([[-2,4,-6,8],[-2,0,1,2]])).share(alice,bob)

In [3]:
(x > y).get()


 1  0  1  0
 1  0  0  0
[syft.core.frameworks.torch.tensor.LongTensor of size 2x4]

In [4]:
(x < y).get()


 0  1  0  1
 0  0  0  0
[syft.core.frameworks.torch.tensor.LongTensor of size 2x4]

In [5]:
(x >= y).get()


 1  0  1  0
 1  1  1  1
[syft.core.frameworks.torch.tensor.LongTensor of size 2x4]

In [6]:
(x <= y).get()


 0  1  0  1
 0  1  1  1
[syft.core.frameworks.torch.tensor.LongTensor of size 2x4]

In [7]:
(x == y).get()


 0  0  0  0
 0  1  1  1
[syft.core.frameworks.torch.tensor.LongTensor of size 2x4]

In [5]:
v = sy.FloatTensor([1.1])
v2 = sy.FloatTensor([1.2])
v.__gt__(v2)


 0
[syft.core.frameworks.torch.tensor.ByteTensor of size 1]

In [3]:
# def newgt(self, *args, **kwargs):
#     try:
#         out = self.child > args[0].child
#         return out
#     except:
#         return self.native___gt__(*args, **kwargs)

In [4]:
# # torch.LongTensor.native___gt__ = torch.LongTensor.__gt__
# torch.LongTensor.__gt__ = newgt

In [5]:
x > y

RuntimeError: Command "gt" is not a supported Torch operation.

In [5]:
x.child > y.child

executing
running relu
running relu


[Head of chain]
[syft.core.frameworks.torch.tensor.LongTensor with no dimension]

In [6]:
(x.child > y.child).get()

executing
running relu
running relu



 1  0  1  0
 1  0  0  0
[syft.core.frameworks.torch.tensor.LongTensor of size 2x4]

In [8]:
out = x.relu()

running relu
running relu


In [6]:
out.get()


 0  3  0  7
 0  0  1  2
[syft.core.frameworks.torch.tensor.LongTensor of size 2x4]